### Import data_utils

In [1]:
from climsim_utils.data_utils import *

2023-08-20 18:44:56.401614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 18:44:57.390662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Instantiate class

In [2]:
grid_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/grid_info/ClimSim_low-res_grid-info.nc'
norm_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/normalizations/'

grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + 'inputs/input_mean.nc')
input_max = xr.open_dataset(norm_path + 'inputs/input_max.nc')
input_min = xr.open_dataset(norm_path + 'inputs/input_min.nc')
output_scale = xr.open_dataset(norm_path + 'outputs/output_scale.nc')

data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale)

### Load data and set pressure grid

In [3]:
# paths to scoring data
input_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/scoring_input.npy'
target_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/scoring_target.npy'

# paths to model predictions
cvae_pred_path = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/cVAE/cvae_preds_manual.h5'
hsr_pred_path = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/HSR/hsr_preds_bestcrps.h5'
rpn_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy'
cnn_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy'
mlp_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy'

# set variables to V1 subset
data.set_to_v1_vars()

# path to target input
data.input_scoring = np.load(input_path)

# path to target output
data.target_scoring = np.load(target_path)

# set pressure weights
data.set_pressure_grid(data.input_scoring)

# load model predictions
data.model_names = ['CNN','cVAE','HSR','MLP', 'RPN']
preds = [data.load_npy_file(load_path = cnn_pred_path), 
         data.load_h5_file(load_path = cvae_pred_path), 
         data.load_h5_file(load_path = hsr_pred_path), 
         data.load_npy_file(load_path = mlp_pred_path), 
         data.load_npy_file(load_path = rpn_pred_path)]
data.preds_scoring = dict(zip(data.model_names, preds))


### Weight outputs

1. Undo output scaling

2.  Weight vertical levels by dp/g

3. Weight horizontal area of each grid cell by a[x]/mean(a[x])

4. Convert units to a common energy unit

In [4]:
data.reweight_target(data_split = 'scoring')
data.reweight_preds(data_split = 'scoring')

### Calculate metrics

In [5]:
data.metrics_names = ['MAE', 'RMSE', 'R2']
data.create_metrics_df(data_split = 'scoring')

AssertionError: 

In [ ]:
hmm = data.preds_scoring['MLP']

In [ ]:
wow = data.preds_scoring["MLP"]['heating']
wow2 = data.preds_scoring["MLP"]["netsw"]

In [ ]:
np.atleast_1d(wow.mean(axis = (0,1))).shape

In [ ]:
np.atleast_1d(wow2.mean(axis = (0,1))).shape[0]

In [ ]:
for name in data.preds_scoring:
    print(name)

In [ ]:
hmm['heating'].shape

In [ ]:
hmm['netsw'].shape

In [ ]:
columns = ['A', 'B', 'C']
index = ['X', 'Y', 'Z']
df = pd.DataFrame(columns=columns, index=index)
df.index.name = 'Variable'
df

In [ ]:
columns = ['A', 'B', 'C']
index = range(12)
df = pd.DataFrame(columns=columns, index=index)
df.index.name = 'output_idx'
df

In [ ]:
df.loc[4:7,'A'] = np.arange(4,8)

In [ ]:
df

In [ ]:
data.calc_MAE(hmm['heating'], data.target_scoring['heating'])

In [ ]:
data.target_scoring

In [ ]:
for var in hmm:
    print(var)

In [ ]:
hmm['heating'].mean(axis = (0,1)).shape

In [ ]:
tuple(range(hmm['heating'].ndim-1))

In [ ]:
scoring_data.preds_scoring['CNN'].shape

In [ ]:

cnn_ex = scoring_data.calc_MAE(scoring_data.preds_scoring['CNN'], scoring_data.target_scoring)

In [ ]:
scoring_data.preds_scoring['CNN'].shape

## Time to compare

### Weighted output values

In [ ]:
assert(np.sum(DS_ENERGY['true']['ptend_t'].values - heating) == 0)
assert(DS_ENERGY['true']['ptend_t'].values.shape == heating.shape)

assert(np.sum(DS_ENERGY['true']['ptend_q0001'].values - moistening) == 0)
assert(DS_ENERGY['true']['ptend_q0001'].values.shape == moistening.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_NETSW'].values - netsw) == 0)
assert(DS_ENERGY['true']['cam_out_NETSW'].values.shape == netsw.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_FLWDS'].values - flwds) == 0)
assert(DS_ENERGY['true']['cam_out_FLWDS'].values.shape == flwds.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_PRECSC'].values - precsc) == 0)
assert(DS_ENERGY['true']['cam_out_PRECSC'].values.shape == precsc.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_PRECC'].values - precc) == 0)
assert(DS_ENERGY['true']['cam_out_PRECC'].values.shape == precc.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLS'].values - sols) == 0)
assert(DS_ENERGY['true']['cam_out_SOLS'].values.shape == sols.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLL'].values - soll) == 0)
assert(DS_ENERGY['true']['cam_out_SOLL'].values.shape == soll.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLSD'].values - solsd) == 0)
assert(DS_ENERGY['true']['cam_out_SOLSD'].values.shape == solsd.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLLD'].values - solld) == 0)
assert(DS_ENERGY['true']['cam_out_SOLLD'].values.shape == solld.shape)

### Metrics

In [ ]:
heating_mlp, moistening_mlp, netsw_mlp, flwds_mlp, precsc_mlp, precc_mlp, sols_mlp, soll_mlp, solsd_mlp, solld_mlp = \
    scoring_data.output_weighting(scoring_data.preds_scoring['MLP'])

In [ ]:
pred = heating_mlp
actual = heating

In [ ]:
sq_diff = (pred - actual)**2
var_time = (actual - actual.mean(axis = 0)[np.newaxis, :, :])**2
(1 - sq_diff.mean(axis = 0)/var_time.mean(axis = 0)).mean(axis = 0).shape

In [ ]:
sq_diff.shape

In [ ]:
var_time.shape

In [ ]:
sq_diff[..., None].shape

In [ ]:
scoring_data.latlonnum

In [ ]:
heating_mlp.shape

In [ ]:
len(heating_mlp.shape)

In [ ]:
assert heating_mlp.shape == (4380, 384, 60) or len(heating_mlp.shape) == 2

In [ ]:
heating.sum(axis = 0).shape

In [ ]:
heating.mean(axis = 0)[np.newaxis, :, :].shape

##### MAE

In [ ]:
np.mean(np.abs(heating_mlp - heating))

In [ ]:
hmm = np.abs(heating_mlp - heating).mean(axis = (0,1))

In [ ]:
np.mean(hmm)

In [ ]:
np.mean(np.abs(moistening_mlp - moistening))

In [ ]:
np.mean(np.abs(netsw_mlp - netsw))

In [ ]:
np.mean(np.abs(flwds_mlp - flwds))

In [ ]:
np.mean(np.abs(precsc_mlp - precsc))

In [ ]:
np.mean(np.abs(precc_mlp - precc))

In [ ]:
np.mean(np.abs(sols_mlp - sols))

In [ ]:
np.mean(np.abs(soll_mlp - soll))

In [ ]:
np.mean(np.abs(solsd_mlp - solsd))

In [ ]:
np.mean(np.abs(solld_mlp - solld))

##### RMSE

In [ ]:
np.sqrt(np.mean((heating_mlp - heating)**2))

In [ ]:
np.sqrt(np.mean((moistening_mlp - moistening)**2))

In [ ]:
np.sqrt(np.mean((precsc_mlp - precsc)**2))

In [ ]:
np.sqrt(np.mean((solld_mlp - solld)**2))